In [1]:
import os
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import default_data_collator, DataCollatorForLanguageModeling, AutoTokenizer
from datasets import load_from_disk
from datasets import DatasetDict
from transformers import AutoConfig
from torch.optim import AdamW  # ✅ 이걸로 교체


import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from torch import nn
from datasets import load_dataset
import random
import re

/home/woogoonkyu/anaconda3/envs/lora_null/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Load

In [2]:
import datasets


from datasets import load_from_disk
data = load_from_disk('/nas_homes/projects/rapa/Dataset/data_updated')

In [3]:
data

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'source'],
        num_rows: 42782
    })
    test: Dataset({
        features: ['output', 'input', 'source'],
        num_rows: 10699
    })
})

In [4]:
print(data['train']['input'][0]), print(data['train']['output'][0])

다른 약물과의 상호작용을 고려하여 콜레스테롤 관리를 위해 프라바스타틴을 복용할 때 발생할 수 있는 부작용에 대해 자세히 설명하세요. 또한 누가 이 약을 복용하지 말아야 하는지 설명하세요.

***Answer:
프라바스타틴은 콜레스테롤 수치를 관리하는 데 일반적으로 사용되는 약물입니다. 다른 약물과 마찬가지로 잠재적인 부작용이 있을 수 있으며 다른 약물과 상호작용할 수도 있습니다.

프라바스타틴의 가장 흔한 부작용으로는 두통, 근육통 또는 쇠약, 복통, 변비, 설사, 메스꺼움, 가벼운 피부 발진 등이 있습니다. 이러한 부작용은 대개 일시적이며 신체가 약물에 적응하면서 저절로 해결될 수 있습니다. 그러나 심각하거나 지속적인 부작용이 발생하면 의료진에게 문의하는 것이 중요합니다.

약물 상호작용의 경우, 잠재적인 부작용을 피하기 위해 일반 의약품, 보충제, 허브 제품을 포함하여 복용 중인 모든 약물을 의료진에게 알려야 합니다. 특정 약물은 프라바스타틴과 상호작용하여 부작용 위험을 높이거나 효과를 감소시킬 수 있습니다.

예를 들어, 다음과 같은 약물과 함께 복용할 경우 프라바스타틴을 피하거나 주의해서 사용해야 합니다.
1. 사이클로스포린: 이 면역 억제제는 프라바스타틴의 혈중 농도를 증가시켜 부작용 위험을 증가시킬 수 있습니다.
2. 젬피브로질: 젬피브로질과 프라바스타틴을 함께 복용하면 횡문근융해증과 같은 심각한 근육 문제가 발생할 위험이 높아질 수 있습니다.
3. 와파린: 프라바스타틴은 와파린과 같은 항응고제의 효과를 높여 출혈의 위험을 증가시킬 수 있습니다.

잠재적으로 상호작용할 수 있는 다른 약물이 있을 수 있습니다.
프라바스타틴은 콜레스테롤 수치를 관리하는 데 일반적으로 사용되는 약물입니다. 다른 약물과 마찬가지로 잠재적인 부작용이 있을 수 있으며 다른 약물과 상호작용할 수도 있습니다.

프라바스타틴의 가장 흔한 부작용으로는 두통, 근육통 또는 쇠약, 복통, 변비, 설사, 메스꺼움, 가벼운 피부 발진 등이 있습니다. 이러한 부작용은 대개 일시적이며 신체가 

(None, None)

In [5]:
print(data['train']['output'][0])

프라바스타틴은 콜레스테롤 수치를 관리하는 데 일반적으로 사용되는 약물입니다. 다른 약물과 마찬가지로 잠재적인 부작용이 있을 수 있으며 다른 약물과 상호작용할 수도 있습니다.

프라바스타틴의 가장 흔한 부작용으로는 두통, 근육통 또는 쇠약, 복통, 변비, 설사, 메스꺼움, 가벼운 피부 발진 등이 있습니다. 이러한 부작용은 대개 일시적이며 신체가 약물에 적응하면서 저절로 해결될 수 있습니다. 그러나 심각하거나 지속적인 부작용이 발생하면 의료진에게 문의하는 것이 중요합니다.

약물 상호작용의 경우, 잠재적인 부작용을 피하기 위해 일반 의약품, 보충제, 허브 제품을 포함하여 복용 중인 모든 약물을 의료진에게 알려야 합니다. 특정 약물은 프라바스타틴과 상호작용하여 부작용 위험을 높이거나 효과를 감소시킬 수 있습니다.

예를 들어, 다음과 같은 약물과 함께 복용할 경우 프라바스타틴을 피하거나 주의해서 사용해야 합니다.
1. 사이클로스포린: 이 면역 억제제는 프라바스타틴의 혈중 농도를 증가시켜 부작용 위험을 증가시킬 수 있습니다.
2. 젬피브로질: 젬피브로질과 프라바스타틴을 함께 복용하면 횡문근융해증과 같은 심각한 근육 문제가 발생할 위험이 높아질 수 있습니다.
3. 와파린: 프라바스타틴은 와파린과 같은 항응고제의 효과를 높여 출혈의 위험을 증가시킬 수 있습니다.

잠재적으로 상호작용할 수 있는 다른 약물이 있을 수 있습니다.


In [6]:
print(data['train']['input'][1].split('***Answer:')[1])


의대 입학부터 의사 면허 취득까지의 과정은 여러 단계로 나눌 수 있습니다:

1. 학부 교육: 첫 번째 단계는 과학 또는 의학 관련 분야에서 학사 학위를 이수하는 것입니다. 이를 통해 의과대학에 필요한 기초 지식을 습득할 수 있습니다.

2. 의과대학 입학 시험(MCAT): 의대 지망생은 학부 교육을 마친 후 MCAT를 치러야 합니다. 이 표준화된 시험은 과학적 개념에 대한 지식과 비판적 사고 능력을 평가합니다.

3. 의과 대학 지원: 다음 단계는 의과대학에 지원하는 것입니다. 학생들은 일반적으로 미국 의과대학 지원 서비스(AMCAS)를 통해 지원서를 제출합니다. 지원서에는 자기소개서, 추천서, 성적증명서가 포함됩니다.

4. 의과대학: 의대에 입학하면 학생들은 4년간의 엄격한 교육을 받게 됩니다. 커리큘럼에는 강의실 강의, 실험실 작업, 다양한 전문 분야에서 실습 경험을 쌓을 수 있는 임상 순환이 포함됩니다.

5. 미국 의사 면허 시험(USMLE): 의과 대학 재학 중 학생들은 기초 과학 지식을 테스트하는 USMLE 1단계 시험을 통과해야 합니다. 그 후 임상 로테이션 기간 동안 2단계 임상 지식(CK) 및 2단계 임상 술기(CS) 시험에도 합격해야 합니다.

6. 레지던트 수련: 의과대학을 졸업하면 졸업생은 레지던트 프로그램에 들어갑니다. 이 프로그램은 선택한 전문 분야에 따라 3년에서 7년까지 지속됩니다. 레지던트는 숙련된 의사의 감독 하에 일하면서 더 많은 실무 경험과 전문 교육을 받습니다.

7.


In [7]:
print(data['train']['output'][2])

방사선학 보고서는 방사선과 전문의가 수행한 영상 검사에 대한 상세한 서면 해석입니다. 여기에는 다음 구성 요소가 포함됩니다.

1. 환자 정보: 이 섹션에는 이름, 나이, 성별, 의료 기록 번호와 같은 환자의 식별 정보가 포함됩니다.

2. 연구 세부 정보: X-레이, MRI, CT 스캔 또는 초음파 등 수행한 영상 검사 유형에 대한 정보를 제공합니다. 보고서에는 검사 날짜 및 시간도 언급됩니다.

3. 기술: 이 섹션에서는 사용된 장비, 영상 매개변수 및 투여된 조영제 등 영상 검사가 수행된 방법에 대한 기술적 세부 사항을 설명합니다.

4. 결과: 방사선 전문의가 검사에서 얻은 이미지를 해석하는 보고서의 주요 부분입니다. 여기에는 확인된 이상 또는 질병의 크기, 모양, 위치 및 특성을 포함하여 정상 또는 비정상 소견을 자세히 설명합니다. 또한 확인된 소견과 관련된 측정 또는 계산을 제공할 수도 있습니다.

5. 인상: 이 섹션에서는 주요 결과를 요약하고 연구에 대한 전반적인 해석을 제공합니다. 여기에는 관찰된 결과를 설명할 수 있는 가능한 질환의 목록인 감별 진단이 포함될 수 있습니다.

6. 권고: 방사선 전문의는 검사 결과에 따라 진단을 확인하거나 명확히 하는 데 도움이 되는 추가 진단 검사 또는 절차를 제안할 수 있습니다. 또한 후속 영상 검사를 권장하거나 추가 평가 또는 치료를 위해 환자를 다른 전문의에게 의뢰할 수도 있습니다.


# Instruction 추가

In [8]:
instruction = """Please read the following content and respond as a licensed medical doctor with expertise in the relevant field. Provide a detailed, accurate, and professional Answer. Please answer in korean. \n***Content:\n"""

In [9]:
from datasets import load_from_disk
import torch
from transformers import AutoTokenizer
from datasets import DatasetDict
import random

# 모델에 맞는 tokenizer 로딩


tokenizer = AutoTokenizer.from_pretrained("aaditya/Llama3-OpenBioLLM-8B")

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 데이터셋 로딩
dataset = data

def preprocess_function(example):
    prompt = instruction + example["input"]
    answer = example["output"]
    answer_tag = "***Answer:\n"
    
    # answer_tag 없으면 교체
    if answer_tag not in prompt:
        if "Answer:" in prompt:
            prompt = prompt.replace("Answer:", answer_tag)
        else:
            prompt +=  answer_tag
            

    # 전체 입력 문장


    prompt = prompt.replace("***Answer:***Answer:", "***Answer:") 

    # 1. 🪓 텍스트 기준으로 분리
    prompt_part, answer_part = prompt.split(answer_tag, 1)

    # 2. 🧼 정리된 텍스트
    prompt_text = prompt_part + answer_tag  # 태그까지 포함
    answer_text = answer_part.strip()       # 혹시 앞뒤 공백 제거

    # 3. ✂️ 각각 토크나이즈
    prompt_tokenized = tokenizer(prompt_text, add_special_tokens=False)
    answer_tokenized = tokenizer(answer_text, add_special_tokens=False)

    # 4. 🧩 전체 input_ids + attention_mask
    input_ids = prompt_tokenized["input_ids"] + answer_tokenized["input_ids"]
    attention_mask = [1] * len(input_ids)

    # 5. 🔐 labels: prompt 구간은 -100, answer 구간은 그대로
    labels = [-100] * len(prompt_tokenized["input_ids"]) + answer_tokenized["input_ids"]

    # 6. 📏 max_length 맞춰 패딩/트렁케이션
    max_length = 512
    input_ids = input_ids[:max_length]
    attention_mask = attention_mask[:max_length]
    labels = labels[:max_length]

    # 7. ⛑️ 패딩
    pad_id = tokenizer.pad_token_id or tokenizer.eos_token_id
    pad_len = max_length - len(input_ids)
    input_ids += [pad_id] * pad_len
    attention_mask += [0] * pad_len
    labels += [-100] * pad_len

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }


In [10]:
tokenizer = AutoTokenizer.from_pretrained("aaditya/Llama3-OpenBioLLM-8B")

num = 0
name_lst = []


if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
name = tokenizer.name_or_path.split('/')[0].replace('-','_')

# 데이터셋 전체에 전처리 적용
tokenized_datasets = dataset.map(preprocess_function, batched=False)

# 원본 텍스트 열 제거
tokenized_datasets = tokenized_datasets.remove_columns(["input", "output", "source"])

# 디스크에 저장
if name in name_lst:
    name = name + str(num)
    num += 1
else:
    pass

name_lst.append(name)


Map: 100%|██████████| 10699/10699 [00:14<00:00, 748.24 examples/s] 


In [11]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 42782
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10699
    })
})


In [12]:
print(data['train']['output'][0])

프라바스타틴은 콜레스테롤 수치를 관리하는 데 일반적으로 사용되는 약물입니다. 다른 약물과 마찬가지로 잠재적인 부작용이 있을 수 있으며 다른 약물과 상호작용할 수도 있습니다.

프라바스타틴의 가장 흔한 부작용으로는 두통, 근육통 또는 쇠약, 복통, 변비, 설사, 메스꺼움, 가벼운 피부 발진 등이 있습니다. 이러한 부작용은 대개 일시적이며 신체가 약물에 적응하면서 저절로 해결될 수 있습니다. 그러나 심각하거나 지속적인 부작용이 발생하면 의료진에게 문의하는 것이 중요합니다.

약물 상호작용의 경우, 잠재적인 부작용을 피하기 위해 일반 의약품, 보충제, 허브 제품을 포함하여 복용 중인 모든 약물을 의료진에게 알려야 합니다. 특정 약물은 프라바스타틴과 상호작용하여 부작용 위험을 높이거나 효과를 감소시킬 수 있습니다.

예를 들어, 다음과 같은 약물과 함께 복용할 경우 프라바스타틴을 피하거나 주의해서 사용해야 합니다.
1. 사이클로스포린: 이 면역 억제제는 프라바스타틴의 혈중 농도를 증가시켜 부작용 위험을 증가시킬 수 있습니다.
2. 젬피브로질: 젬피브로질과 프라바스타틴을 함께 복용하면 횡문근융해증과 같은 심각한 근육 문제가 발생할 위험이 높아질 수 있습니다.
3. 와파린: 프라바스타틴은 와파린과 같은 항응고제의 효과를 높여 출혈의 위험을 증가시킬 수 있습니다.

잠재적으로 상호작용할 수 있는 다른 약물이 있을 수 있습니다.


In [13]:
from tqdm.auto import tqdm

def find_invalid_samples(example, idx):
    input_ids = example["input_ids"]
    masked_labels = example["labels"]

    # 기본적으로 길이 체크
    if len(input_ids) != len(masked_labels):
        return {"invalid": True}

    # masked_labels의 유효 시작점 찾기
    try:
        masked_label_start_idx = next(i for i, v in enumerate(masked_labels) if v != -100)
    except StopIteration:
        return {"invalid": True}

    # input_ids와 masked_labels 시작점 비교
    if input_ids[masked_label_start_idx] != masked_labels[masked_label_start_idx]:
        return {"invalid": True}

    # 정상인 경우
    return {"invalid": False}

# 각 split별로 문제 인덱스 찾기
invalid_indices = {}

for split in ["train", "test"]:
    result_dataset = tokenized_datasets[split].map(
        find_invalid_samples,
        with_indices=True,
        batched=False,
        remove_columns=tokenized_datasets[split].column_names
    )

    # 문제가 있는 샘플의 인덱스만 추출
    invalid_idx = [i for i, example in tqdm(enumerate(result_dataset), total=len(result_dataset)) if example["invalid"]]
    invalid_indices[split] = invalid_idx

# 최종 결과 출력
# print("🚨 문제가 있는 샘플 인덱스 결과:")
# print(invalid_indices)


100%|██████████| 10699/10699 [00:00<00:00, 125877.24it/s]


In [14]:
def check_preprocessed_sample(sample, tokenizer):
    input_ids = sample["input_ids"]
    labels = sample["labels"]

    # 🔍 전체 디코딩 (input_ids)
    decoded_input = tokenizer.decode(input_ids, skip_special_tokens=True)

    # 🔍 정답 디코딩 (labels 중 -100 제외)
    label_ids = [t for t in labels if t != -100]
    decoded_label = tokenizer.decode(label_ids, skip_special_tokens=True)

    # 🔢 시각적으로 보기 좋게 정리
    print("=" * 60)
    print("🧾 Full input (input_ids):")
    print(decoded_input)
    print("-" * 60)
    print("🎯 Target only (labels):")
    print(decoded_label)
    print("=" * 60)
    print()

In [15]:
from tqdm.auto import tqdm

def is_valid_sample(example):
    input_ids = example["input_ids"]
    masked_labels = example["labels"]

    # 길이 체크
    if len(input_ids) != len(masked_labels):
        return False

    # masked_labels의 유효 시작점 찾기
    try:
        masked_label_start_idx = next(i for i, v in enumerate(masked_labels) if v != -100)
    except StopIteration:
        return False

    # 시작점 비교
    if input_ids[masked_label_start_idx] != masked_labels[masked_label_start_idx]:
        return False

    # ✅ 모든 라벨이 -100인지 체크
    if all(v == -100 for v in masked_labels):
        return False

    return True

# 문제가 있는 샘플을 제거한 새로운 데이터셋 만들기
filtered_datasets = DatasetDict()

for split in ["train", "test"]:
    filtered_datasets[split] = tokenized_datasets[split].filter(
        is_valid_sample,
        num_proc=4,
    )

# 결과 출력
print(filtered_datasets)


Filter (num_proc=4): 100%|██████████| 10699/10699 [00:01<00:00, 7821.59 examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 33298
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8327
    })
})


# Save Dataset

In [16]:
# filtered_datasets.save_to_disk('/nas_homes/projects/rapa/Dataset/filtered_updated_datasets_512_instruction')

Saving the dataset (1/1 shards): 100%|██████████| 8327/8327 [00:04<00:00, 1717.55 examples/s] 
